In [1]:
import pandas as pd
import numpy as np
import markovify
import torch
import matplotlib.pyplot as plt
from ChordLanguageModel import ChordLanguageModel

In [2]:
CHAIN_ORDER = 3
N_CANDIDATES = 50000
N_BARS = 8
DATA_PATH = "chords3.csv"
rnn_config = {
    "HIDDEN_DIM": 100,
    "EMBED_DIM": 250,
    "DROPOUT": 0.1,
    "L2": 0.0005,
    "LEARNING_RATE": 0.001,
    "BATCH_SIZE": 16,
    "N_LAYERS": 1,
    "N_EPOCHS" :70
}

In [ ]:
clm = ChordLanguageModel(DATA_PATH)
clm.train_model(rnn_config)

784
197


/Users/lauri/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:57: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


EPOCH 0: tensor(259.0896, grad_fn=<AddBackward0>)
TEST LOSS: tensor(4.2090)
EPOCH 1: tensor(193.6254, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.8799)
EPOCH 2: tensor(180.4180, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.7089)
EPOCH 3: tensor(171.9788, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.5993)
EPOCH 4: tensor(165.4918, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.5240)
EPOCH 5: tensor(160.6025, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.4606)
EPOCH 6: tensor(156.9107, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.4159)
EPOCH 7: tensor(153.5896, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.3761)
EPOCH 8: tensor(150.4122, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.3477)
EPOCH 9: tensor(147.7833, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.3245)
EPOCH 10: tensor(145.4734, grad_fn=<AddBackward0>)
TEST LOSS: tensor(3.3095)


In [ ]:
df = pd.read_csv(DATA_PATH)
df.chords = df.chords.apply(lambda x: eval(x))
mc_data = df.chords.to_numpy().tolist()
mc = markovify.Chain(mc_data, state_size=CHAIN_ORDER)

In [ ]:
def validate_time(c):
    bars = 0
    time = 0
    fail = False
    for chord in c:
        time += 1/int(chord[0])
        if time == 1:
            bars = bars + 1
            time = 0
            continue
        if time > 1:
            fail = True
            break
    if not fail:
        if bars == N_BARS:
            return True
    else:
        return False
def validate_walk_length(c):
    if len(c) == N_BARS:
        return True
candidates = []
for i in range(N_CANDIDATES):
    c = mc.walk()
    candidates.append(c)
accepted = []
for candidate in candidates:
    if validate_walk_length(candidate):
        accepted.append(candidate)
accepted = [[c[1] for c in cp] for cp in accepted]
#accepted

In [ ]:
scores = clm.evaluate(accepted)
#scores

In [ ]:
scored = []
for i in range(len(accepted)):
    scored.append((accepted[i], scores[i]))
scored.sort(key=lambda x: x[1], reverse=True)
best = scored[:20]
best

In [ ]:
plt.hist(scores, bins = 50)
plt.show